# Computer vision

### Important imports

In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Check if GPU acceleration is enabled
if tf.test.is_gpu_available():
    print('GPU acceleration is enabled.')
else:
    print('GPU acceleration is not available. Training will be performed on CPU.')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU acceleration is not available. Training will be performed on CPU.


### Loading data

In [3]:
# Load the FER2013 dataset using pandas
data = pd.read_csv(r"C:\Users\B.N.KULKARNI\Downloads\archive.zip")


### Preprocessing and Augmentation

In [ ]:
#The preprocess_image function takes the path to an image, reads it using OpenCV, converts it to grayscale, resizes it to a fixed size (in this case, 48x48), and normalizes the pixel values between 0 and 1.

#Preprocessing function
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (48, 48))
    image = image / 255.0
    return image

#We specify the path to the dataset folder and load the image files. Then, we preprocess each image using the preprocess_image function and store the preprocessed images in a list. Next, we convert the list of images to a NumPy array (X). Finally, we reshape the array to match the expected input shape of the model.

dataset_path = 'path/to/dataset/folder'

# Load the image files from the dataset folder
image_files = [os.path.join(dataset_path, file) for file in os.listdir(dataset_path)]

# Preprocess the images and store them in a list
images = [preprocess_image(file) for file in image_files]

# Convert the list of images to a NumPy array
X = np.array(images)

# Reshape the input array to match the expected input shape of the model
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

# Split dataset into training, validation, and test sets
train_data, val_data, train_labels, val_labels = train_test_split(X['pixels'], X['emotion'], test_size=0.2, random_state=42)

# Perform data augmentation on the training dataset
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
datagen.fit(X)

### Build the emotion detection model

In [ ]:
#We build a simple convolutional neural network (CNN) model using the Sequential API from Keras. The model consists of several convolutional layers followed by max pooling layers, a flatten layer, and fully connected (dense) layers. The output layer has 7 units corresponding to the 7 emotions (labels) we want to detect. We compile the model with the Adam optimizer, sparse categorical cross-entropy loss, and accuracy metric.

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Train the emotion detection model

In [ ]:
# Specify the number of training epochs and batch size
epochs = 10
batch_size = 32

# Train the model
model.fit(datagen.flow(X, y, batch_size=batch_size), epochs=epochs)


### Test on the Test Dataset

In [ ]:
#Load the true labels for the test data and evaluate the model on the test data and calculate accuracy

test_dataset_path = 'path/to/test/dataset/folder'

# Load the image files from the test dataset folder
test_image_files = [os.path.join(test_dataset_path, file) for file in os.listdir(test_dataset_path)]

# Preprocess the test images and store them in a list
test_images = [preprocess_image(file) for file in test_image_files]

# Convert the list of test images to a NumPy array
X_test = np.array(test_images)

# Reshape the test input array to match the expected input shape of the model
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


### Save the model

In [ ]:
#Use the save() method of the model to save it to a specified file path. Replace 'path/to/save/model.h5' with the desired location and filename for your saved model. This will save the entire model including its architecture, weights, and optimizer state.

# Specify the path to save the model
model_path = 'path/to/save/model.h5'

# Save the model
model.save(model_path)
print("Model saved successfully.")


### Make predictions on new images

In [ ]:
#The predict_emotion function takes the path to a new image, preprocesses it, and performs the prediction using the trained model. It returns the predicted emotion label.

from tensorflow.keras.models import load_model

# Specify the path to the saved model
saved_model_path = 'path/to/saved/model.h5'

# Load the saved model
loaded_model = load_model(saved_model_path)
print("Model loaded successfully.")

# Load and preprocess a new image
def predict_emotion(image_path):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)
    image = image.reshape(image.shape[0], image.shape[1], image.shape[2], 1)
    predicted_probs = model.predict(image)[0]
    emotion_label = np.argmax(predicted_probs)
    return emotion_label

# Make predictions using the loaded model
predicted_probs = loaded_model.predict(new_image)[0]
emotion_label = np.argmax(predicted_probs)

# Print the predicted emotion label
print("Predicted Emotion Label:", emotion_label)


##### The architecture of the CNN model used in the example is a simple and commonly used architecture for image classification tasks. It consists of a series of convolutional and pooling layers followed by fully connected layers.

Here's a breakdown of the chosen architecture and the reasoning behind it:

1. **Convolutional Layers:**
   - The first convolutional layer has 32 filters with a kernel size of 3x3. This layer is responsible for learning low-level features from the input images.
   - The second convolutional layer has 64 filters with a kernel size of 3x3. This layer captures higher-level features by building upon the features learned in the previous layer.
   - The third convolutional layer has 128 filters with a kernel size of 3x3. It further extracts more complex and abstract features from the input images.

2. **Pooling Layers:**
   - Max pooling layers with a pool size of 2x2 are used after each convolutional layer. They help reduce the spatial dimensions and capture the most important features.

3. **Flatten Layer:**
   - The flatten layer is used to flatten the 3D feature maps into a 1D vector, which can be fed into the fully connected layers.

4. **Fully Connected Layers:**
   - The first fully connected layer consists of 64 neurons with ReLU activation. It learns complex patterns and representations based on the extracted features.
   - The final output layer consists of `num_classes` neurons (7 in the case of FER2013 dataset) with softmax activation, producing the predicted probabilities for each emotion class.

This architecture strikes a balance between model complexity and simplicity. It captures both low-level and high-level features from the input images, gradually learning more abstract representations. The number of filters and neurons in the fully connected layers can be adjusted based on the complexity of the dataset and desired model capacity.

It's important to note that this architecture can be further optimized and refined based on the specific requirements and constraints of the project. Different architectures, such as deeper or wider networks, can also be explored to potentially improve the accuracy of the model.

# NLP

##### To build an emotion detection model for unprocessed text paragraphs, we can utilize natural language processing (NLP) techniques and deep learning algorithms. One popular approach is to use recurrent neural networks (RNNs) or transformer-based models such as BERT. Here's an outline of the code and approach, along with code snippets, for building such a model:

1. **Dataset Selection:**
  - A suitable dataset for emotion detection in text paragraphs is the "ISEAR" dataset, which contains labeled emotional texts. You can download it from this link: https://www.unige.ch/cisa/research/materials-and-online-research/research-material/
   
2. **Data Preprocessing:**
   - Load the dataset and perform necessary preprocessing steps such as removing special characters, lowercasing, tokenization, and splitting the data into training and testing sets.

3. **Word Embedding:**
   - Convert the text data into numerical vectors using word embeddings like Word2Vec, GloVe, or FastText. These embeddings capture semantic relationships between words.
   - You can either train your own word embeddings on your dataset or use pre-trained embeddings that are readily available.

4. **Model Architecture:**
   - Build an RNN or transformer-based model using deep learning frameworks like TensorFlow or PyTorch. These models excel in capturing sequential dependencies and long-term dependencies in text data.

5. **Model Training:**
   - Train the model using the training data and optimize it with techniques like mini-batch gradient descent and backpropagation.
   - Monitor the training process by evaluating the loss and accuracy on the validation set. Adjust hyperparameters if necessary.

6. **Model Evaluation:**
   - Evaluate the trained model on the test set to measure its performance and accuracy.
   - Calculate metrics such as accuracy, precision, recall, and F1-score to assess the model's ability to detect emotions correctly.

7. **Prediction on New Text:**
   - Use the trained model to predict emotions on new, unseen text paragraphs by preprocessing the text in the same way as the training data and passing it through the trained model.

Now, let's dive into some code snippets to illustrate the steps:





In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Step 1: Load and preprocess the dataset
data = pd.read_csv('path/to/isear_dataset.csv')
# Perform data cleaning and preprocessing if required

# Step 2: Split the dataset into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['text'], data['emotion'], test_size=0.2, random_state=42)

# Step 3: Word Embedding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
vocab_size = len(tokenizer.word_index) + 1
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)
max_sequence_length = max(len(seq) for seq in train_sequences)
train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Step 4: Model Architecture
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

# Step 5: Model Training
model.compile(loss='sparse

_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data, train_labels, validation_split=0.2, epochs=10, batch_size=64)

# Step 6: Model Evaluation
_, accuracy = model.evaluate(test_data, test_labels)
print("Test Accuracy:", accuracy)

# Step 7: Prediction on New Text
new_text = "I am feeling happy today"
new_sequence = tokenizer.texts_to_sequences([new_text])
new_sequence = pad_sequences(new_sequence, maxlen=max_sequence_length)
prediction = model.predict(new_sequence)
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'shame', 'guilt']
predicted_emotion = emotion_labels[prediction.argmax()]
print("Predicted Emotion:", predicted_emotion)

##### In this code snippet, we start by loading the ISEAR dataset and splitting it into training and testing sets. We then perform word embedding using the `Tokenizer` class and pad the sequences to ensure they have the same length. Next, we define the model architecture, which includes an embedding layer, LSTM layer, and dense layers. The model is compiled and trained using the training data. Finally, we evaluate the model on the test set and demonstrate how to make predictions on new, unseen text paragraphs.



##### Preprocessing steps:Certainly! Let's start by understanding the concept of lemmatization and then delve into the preprocessing steps for raw paragraph text.

**Lemmatization:**
Lemmatization is a process in Natural Language Processing (NLP) that reduces words to their base or dictionary form, known as the lemma. The resulting lemma represents the canonical or morphological root form of a word. Lemmatization aims to normalize words, considering their part of speech (POS) tags, to provide a more meaningful representation for downstream NLP tasks.

For example, the lemma of the word "running" is "run," and the lemma of "wolves" is "wolf." By reducing words to their lemmas, we can handle variations of words and treat them as the same token in subsequent analysis.

**Preprocessing Steps for Raw Paragraph Text:**
When working with raw paragraph text data, several preprocessing steps are typically performed to clean and transform the data before feeding it into an emotion detection model. Here are the commonly used preprocessing steps:

1. **Tokenization:** Tokenization involves splitting the text into individual words or tokens. This step breaks down a paragraph into its constituent units to facilitate further analysis.

2. **Lowercasing:** Converting all text to lowercase helps standardize the data and treat words with the same characters as the same token, regardless of their case.

3. **Removal of Special Characters and Punctuation:** Special characters, symbols, and punctuation marks may not provide valuable information for emotion detection. Removing them helps reduce noise and focus on the meaningful content.

4. **Stopword Removal:** Stopwords are common words (e.g., "the," "is," "and") that occur frequently in the text but often carry little semantic significance. Removing stopwords can reduce the dimensionality of the data and speed up processing.

5. **Stemming or Lemmatization:** Stemming and lemmatization aim to reduce words to their root forms. Stemming applies simple rules to truncate words, whereas lemmatization uses linguistic resources and considers the context and part of speech to produce the lemma. Lemmatization generally yields more meaningful results compared to stemming.

6. **Normalization:** Normalization involves additional text transformations such as handling contractions (e.g., converting "can't" to "cannot") or standardizing spellings (e.g., converting "colour" to "color").

7. **Padding and Truncation:** In certain cases, it may be necessary to ensure that all text samples have the same length. Padding involves adding zeros or a special token to make shorter texts the same length as the longest text, while truncation cuts off excessive text to match a predefined length.

These preprocessing steps help transform raw paragraph text into a format that is suitable for training an emotion detection model. The specific steps and their order may vary depending on the nature of the text data and the requirements of the task.

It's essential to analyze the dataset, understand the characteristics of the text, and iteratively refine the preprocessing steps to achieve the best possible representation of the text for the emotion detection task.

In [ ]:
#In this code snippet, we start by importing the necessary modules from the NLTK library. We then download the required NLTK resources for tokenization, stopwords, and lemmatization.Next, we load the raw paragraph text dataset into a Pandas DataFrame (data). The dataset should have a column named 'text' containing the raw text paragraphs.We define a set of stopwords using the NLTK stopwords corpus.The preprocess_text() function is defined to carry out the preprocessing steps for a single text sample. It tokenizes the text using word_tokenize(), converts the tokens to lowercase, removes special characters and punctuation using a list comprehension, removes stopwords, and performs lemmatization using WordNetLemmatizer().The preprocess_text() function is then applied to each text sample in the 'text' column of the DataFrame using the apply() function, and the preprocessed text is stored in a new column named 'preprocessed_text'.Finally, the preprocessed data is split into training and testing sets using train_test_split() as before.Make sure to replace 'path/to/raw_paragraph_text_dataset.csv' with the actual path to your raw paragraph text dataset file.By running this code, you will have the preprocessed text data ready for further processing and training an emotion detection model.

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
data = pd.read_csv('path/to/raw_paragraph_text_dataset.csv')

# Define stopwords
stopwords = set(stopwords.words('english'))

# Function to preprocess a single text sample
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    
    # Removal of special characters and punctuation
    tokens = [token for token in tokens if token not in string.punctuation and token.isalnum()]
    
    # Stopword removal
    tokens = [token for token in tokens if token not in stopwords]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back to a single text string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Preprocess the text data
data['preprocessed_text'] = data['text'].apply(preprocess_text)

# Split the preprocessed data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['preprocessed_text'], data['emotion'], test_size=0.2, random_state=42)


##### Certainly! Word embedding is a technique used in natural language processing (NLP) to represent words in a continuous vector space, where words with similar meanings are closer to each other. It is a form of word representation that captures both semantic and syntactic information about words.

Traditional approaches for representing words in NLP tasks, such as one-hot encoding or count-based methods, treat words as discrete symbols. These methods suffer from high-dimensional and sparse representations, making it difficult to capture the relationships between words and generalize to new, unseen words.

Word embedding techniques, on the other hand, map words to dense vectors of continuous values, often with lower-dimensional representations. These embeddings are learned from large text corpora using unsupervised learning algorithms. The resulting word vectors encode semantic and syntactic relationships between words, allowing for more efficient and meaningful computations.

Here are a few popular word embedding techniques:

1. **Word2Vec:** Word2Vec is a widely used word embedding model that learns continuous word representations based on the distributional hypothesis. It uses a neural network architecture to predict a target word based on its neighboring words in a large corpus. Word2Vec provides dense vector representations that capture semantic relationships between words.

2. **GloVe:** GloVe (Global Vectors for Word Representation) is another popular word embedding model that leverages global word co-occurrence statistics. It constructs a co-occurrence matrix of words and uses matrix factorization techniques to learn word embeddings. GloVe captures both global and local contexts of words, resulting in meaningful representations.

3. **BERT (Bidirectional Encoder Representations from Transformers):** BERT is a transformer-based language model that has revolutionized many NLP tasks. It learns contextualized word representations by considering both left and right contexts of words. BERT is trained on large amounts of unlabeled text data using masked language modeling and next sentence prediction tasks.

Word embeddings offer several benefits in NLP tasks:

- **Semantic Similarity:** Word embeddings allow for measuring semantic similarity between words. Words with similar meanings have embeddings that are closer in the vector space.

- **Word Analogies:** Word embeddings enable algebraic operations on word vectors. For example, `king - man + woman` results in a vector that is close to the embedding of the word `queen`. This allows for analogical reasoning and completion of word analogies.

- **Generalization:** Word embeddings capture semantic and syntactic regularities in language, enabling models to generalize to unseen words or tasks.

To use word embeddings in an emotion detection project, you can leverage pre-trained word embedding models such as Word2Vec or GloVe and incorporate them into your deep learning architecture. These pre-trained word embeddings can be fine-tuned on your specific emotion detection dataset or used as fixed features in the model.

Word embeddings have significantly contributed to the success of various NLP tasks, including sentiment analysis, named entity recognition, machine translation, and more. By leveraging word embeddings, models can better understand and represent the underlying meanings and relationships of words, leading to improved performance in many NLP applications.

Certainly! Word embeddings algorithms, such as Word2Vec and GloVe, aim to learn meaningful representations of words based on their context in a given text corpus. Here's a simplified explanation of the algorithms:

1. **Word2Vec:**
   - Word2Vec uses a neural network architecture, specifically either the Continuous Bag of Words (CBOW) or Skip-gram model.
   - The CBOW model predicts a target word based on its surrounding context words.
   - The Skip-gram model, on the other hand, predicts the surrounding context words given a target word.
   - During training, the model adjusts the weights of the neural network to optimize the prediction of context words or target words.
   - The key idea is that words appearing in similar contexts will have similar word representations.
   - After training, the weights of the neural network are used as the word embeddings, i.e., the dense vector representations of words.

2. **GloVe:**
   - GloVe utilizes the co-occurrence statistics of words in a corpus.
   - It constructs a co-occurrence matrix that captures the frequency of words appearing together in the corpus.
   - The matrix is then factorized using techniques like Singular Value Decomposition (SVD) to obtain lower-dimensional representations.
   - The factorization process results in word embeddings that preserve both global and local contexts.
   - Words that frequently appear together will have similar word embeddings, indicating their semantic relatedness.

Both Word2Vec and GloVe leverage large text corpora to learn word embeddings. The resulting word embeddings are continuous vectors that capture semantic and syntactic relationships between words. Words with similar meanings or appearing in similar contexts will have closer vector representations.

These word embeddings can be further utilized in various natural language processing tasks such as sentiment analysis, text classification, and machine translation. By incorporating pre-trained word embeddings into models, it becomes easier to capture the semantic meaning of words and improve the performance of these tasks.

In summary, word embedding algorithms learn word representations by considering the context or co-occurrence statistics of words in a corpus. These representations enable models to understand the relationships between words and capture their semantic meanings, facilitating more effective natural language processing tasks.

# Performing EDA

In [ ]:
# Just use above model to create two different dataframe of emotion dominant in each photo and text with the text and emotions respectively and finally perform EDA to analyse which emotion results in maximum conversion